В этой тетраде разбиваем данные на n фолдов. Считаем по k эпох для каждого из фолдов и делаем предикшен используя лучшую модель в фолде. Потом результаты осредняем и сабмитим.

In [ ]:
# Здесь нужно получить только индексы для валидионной выборки мы их передадим в фастаи для разделения выборок
kf = 3
val_folds_idx = []
skf = StratifiedKFold(n_splits=kf, random_state=SEED, shuffle=True)
for train_idx, valid_idx in skf.split(train_df_proc.index, train_df_proc.target):
    val_folds_idx.append(valid_idx)
    print(train_idx.shape, valid_idx.shape)

In [ ]:
cols = train_df_proc.columns[1:]

In [ ]:
# Какая то херня с этими табличными моделями, поэтому 
# после каждого фолда сразу делаем предикшен

# Гоняем kf моделей и сохраняем лучшие
predictions = np.zeros((test_df.shape[0]))
for fold in range(kf):
    # Создаем данные для подвыборки
    data = (TabularList.from_df(train_df_proc, path=path_to_data, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(val_folds_idx[fold])
                           .label_from_df(cols=dep_var)
                           .add_test(test, label=0)
                           .databunch())
    data.batch_size = 128
    # Создаем модель 
    learn = tabular_learner(data, 
                            layers=[200,100], 
                            metrics=[accuracy, roc_auc], 
                            emb_drop=0.1, 
                            ps=0.1, 
                            callback_fns=[partial(callbacks.SaveModelCallback, 
                                                 monitor='roc_auc_score', 
                                                 mode='max', 
                                                 name=f'bestmodel_{fold}')])
    
    # Считаем
    learn.fit_one_cycle(2, max_lr=1e-2)
    
    # Считаем сразу предикшн
    p = learn.get_preds(ds_type=DatasetType.Test)
    p_list = p[0][:, 1].tolist()
    predictions += p_list
    
    del data
    del learn
    
# Получаем средний предикшн
predictions /= kf

In [ ]:
make_submition(test_df, learn, 'local_rocauc_83')